In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
df = pd.read_csv('impianti_di_risalita.csv')

In [3]:
comuni = list(set(df['comune']))


In [4]:
for i in range(len(comuni)):
    comuni[i]=comuni[i].replace(' ', '%20')

In [5]:
urls_comuni = []
for comune in comuni:
    urls_comuni.append('https://www.tripadvisor.it/Search?q='+ comune +'&searchSessionId=001362845cba448c.ssid&searchNearby=false&geo=194899&sid=66576BD53D8C33A20E910AAA00F136671686129920162&blockRedirect=true&ssrc=e&rf=5')

In [6]:
data = []

In [7]:
for com in urls_comuni:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(com)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'result-title')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for i in range(30,100,30):

        ristoranti = soup.find_all('div', class_ = 'result-title')

        for ristorante in ristoranti:
            name = ristorante.find('span').text
            onclick_value = ristorante['onclick']
            url = 'https://www.tripadvisor.it' + re.search(r"this, '(.*?)'", onclick_value).group(1)
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_all_elements_located)
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')

            if soup2.find('svg', class_ = 'UctUV d H0'):
                rating = soup2.find('svg', class_ = 'UctUV d H0')['aria-label'].split(' ')[1].strip()
            else:
                rating = 'sconosciuto'
                continue
            if soup2.find('a', class_ = 'dlMOJ'):
                price = soup2.find('a', class_ = 'dlMOJ').text
            else:
                price = 'sconosciuto'
                continue
            if soup2.find_all('span', class_ = 'DsyBj cNFrA'):
                address = soup2.find_all('span', class_ = 'DsyBj cNFrA')[1].text
            else:
                address = 'sconosciuto'
                continue
            if soup2.find_all('span', class_ = 'DsyBj cNFrA'):
                contact = soup2.find_all('span', class_ = 'DsyBj cNFrA')[2].text
            else:
                contact = 'sconosciuto'
                continue

            data.append({
                'name': name,
                'url': url,
                'rating': rating,
                'price': price,
                'address': address,
                'contact': contact
            })

        if soup.find('a', class_ = 'ui_button nav next primary '):
            url = com+f'&o={i}'
            driver.get(url)
        else:
            break

In [8]:
r = pd.DataFrame(data)

In [9]:
r

,name,url,rating,price,address,contact
0,Bar Pasticceria Marika,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,"Corso Milano, 17, 24020 Selvino Italia",+39 393 250 5982
1,Ristorante Sorriso,https://www.tripadvisor.it/Restaurant_Review-g...,"4,5",€€€€,"Via Talpino 79, 24020 Selvino Italia",+39 035 763104
2,Pizzeria Birreria Roma,https://www.tripadvisor.it/Restaurant_Review-g...,"3,5",€€-€€€,"Via Monte Alben, 13, 24020 Selvino Italia",+39 035 763012
3,Ca' di Bore,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€€-€€€,"Via Cantul 38, 24020 Aviatico Italia",+39 035 779167
4,A Casa del Contadino,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€,"Corso Milano, 91, 24020 Selvino Italia",+39 035 764202
...,...,...,...,...,...,...
998,Malga Bassa,https://www.tripadvisor.it/Restaurant_Review-g...,"4,5",€€-€€€,"Via Valle di Scalve 1100, 24020 Onore Italia",+39 329 152 9687
999,Ristorante Pizzeria 4 Pis,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€€-€€€,"Via Placido Piantoni 13, 24020 Colere Italia",+39 0346 54136
1000,Baita Valle Azzurra,https://www.tripadvisor.it/Restaurant_Review-g...,"4,5",€,"Via Valzurio 9 Dopo la chiesa sulla sinistra, ...",+39 0346 28193
1001,Ristorante San Marco,https://www.tripadvisor.it/Restaurant_Review-g...,"4,0",€€-€€€,"Via Pradella 3, 24020 Schilpario Italia",+39 0346 55024


In [1]:
r.to_csv('ristoranti_comuni.csv')

NameError: name 'r' is not defined